# Lesson 6 "Differential Privacy for Deep Learning" project

## Scenario description 


Simple project with toy model

# Project implementation

This project will use the MNIST dataset, provided below.

In [1]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)

In [2]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
priest = sy.VirtualWorker(hook, id="priest")
bob.clear_objects()
alice.clear_objects()
priest.clear_objects()
th.manual_seed(7)

In [3]:
from torch import nn, optim

# A Toy Dataset
#raw_data = th.tensor([[5.,3],[0,8],[9,0],[4,4], [2, 10], [6, 6], [3, 5], [10, 2]])
#raw_target = th.tensor([[15.],[10], [20], [14], [16], [20], [13], [24]])
raw_data = th.rand(1000, 2) * 10

def get_synthetic_target(t):
    return t.mm( th.tensor([[2.], [1]]) ) - 0


raw_target = get_synthetic_target(raw_data)



def normalize_data(t):    
    return t #/ 15

def normalize_target(t):
    return t #/ 42

def denormalize_data(t):
    return t #* 15

def denormalize_target(t):
    return t #* 42

data = raw_data.clone().detach().requires_grad_(True)
target = raw_target.clone().detach().requires_grad_(True)

#print(data)
#print(target)



#data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
#target = th.tensor([[2.],[0], [1], [0]], requires_grad=True)

In [4]:
print(raw_data.owner)
print(data_bob.owner)

<VirtualWorker id:me #objects:0>


NameError: name 'data_bob' is not defined

In [5]:
# A Toy Model
model = nn.Linear(2,1)

opt = optim.SGD(params=model.parameters(), lr=0.00001)

def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

print(model.state_dict())

with th.no_grad():
    prediction = model.forward(data)
    # print(denormalize_target(prediction))
    diff = prediction - target
diff.mean()

tensor(250010.9531)
tensor(15128.1572)
tensor(3650.8193)
tensor(2259.9302)
tensor(1576.2091)
tensor(1111.1749)
tensor(786.9075)
tensor(560.4239)
tensor(402.2028)
tensor(291.6508)
tensor(214.3882)
tensor(160.3725)
tensor(122.5914)
tensor(96.1474)
tensor(77.6209)
tensor(64.6238)
tensor(55.4885)
tensor(49.0499)
tensor(44.4951)
tensor(41.2562)
OrderedDict([('weight', tensor([[1.9423, 0.9749]])), ('bias', tensor([0.4854]))])


tensor(0.0694)

In [6]:
first_half = range(len(data)//2)
second_half = range(len(data)//2, len(data))

data_bob = data[first_half].send(bob)
target_bob = target[first_half].send(bob)

data_alice = data[second_half].send(alice)
target_alice = target[second_half].send(alice)

datasets = [(data_bob, target_bob), (data_alice, target_alice)]



In [7]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.00001)
    
    for iter in range(iterations):
        #print(iter)
        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())
    
    print('model.parameters()', model.state_dict())
    
    with th.no_grad():
        test_data = th.tensor([[1.,1],[0,1],[1,0],[0,0]])
        #test_target = th.tensor([[5.],[3], [4], [2]])
        test_target = get_synthetic_target(test_data)
        
        prediction = model.forward(test_data)
        print('prediction: ', prediction)
        print('actual: ', test_target)
        diff = prediction - test_target
        print('test diff: ', diff)
        
        for _data, _target in datasets:
            model = model.send(_data.location)
            prediction = model.forward(_data)
            #print(prediction)
            diff = prediction - _target
            print('Avg diff: ', diff.get().mean())
            model = model.get()
    
    return model
train()        

tensor(127149.3672, requires_grad=True)
tensor(24089.0430, requires_grad=True)
tensor(6147.5425, requires_grad=True)
tensor(3033.6160, requires_grad=True)
tensor(2333.1245, requires_grad=True)
tensor(1800.7125, requires_grad=True)
tensor(1613.3805, requires_grad=True)
tensor(1271.3636, requires_grad=True)
tensor(1148.0380, requires_grad=True)
tensor(905.3860, requires_grad=True)
tensor(818.4637, requires_grad=True)
tensor(646.0646, requires_grad=True)
tensor(584.5072, requires_grad=True)
tensor(462.0673, requires_grad=True)
tensor(418.4362, requires_grad=True)
tensor(331.4944, requires_grad=True)
tensor(300.5492, requires_grad=True)
tensor(238.8286, requires_grad=True)
tensor(216.8590, requires_grad=True)
tensor(173.0603, requires_grad=True)
tensor(157.4383, requires_grad=True)
tensor(126.3779, requires_grad=True)
tensor(115.2422, requires_grad=True)
tensor(93.2383, requires_grad=True)
tensor(85.2709, requires_grad=True)
tensor(69.7077, requires_grad=True)
tensor(63.9760, requires_grad

Linear(in_features=2, out_features=1, bias=True)

In [8]:
data_bob.location.id

'bob'

In [9]:
def fed_train_models(iterations=20):

    models = {
        'bob': nn.Linear(2,1).send(bob),
        'alice': nn.Linear(2,1).send(alice)
    }
    optims = {
        'bob': optim.SGD(params=models['bob'].parameters(), lr=0.00001),
        'alice': optim.SGD(params=models['alice'].parameters(), lr=0.00001)
    }
    
    for iter in range(iterations):
        print(iter)
        for _data, _target in datasets:
            worker_id = _data.location.id
            model = models[worker_id]
            opt = optims[worker_id]
            
            #print('data location: ', _data.location.id, ' worker_id: ', worker_id)
            
            # do normal training
            opt.zero_grad()            
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            #print('Loss: ', loss.get())
            #print('model.parameters()', model.get().state_dict())
            
    return models
        
models = fed_train_models()       


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [10]:
priest.clear_objects()

from collections import OrderedDict

def agregate_models(models):    
    n = th.tensor(len(models)).send(priest)
    weights = th.zeros(1, 2).send(priest)
    biases = th.zeros(1).send(priest)
    for i, model in enumerate(models):
        sd = model.state_dict()
        w = sd['weight'].move(priest)
        #print('w.shape: ', w.shape)
        weights += w / n
        #print('w: ', w)
        b = sd['bias'].move(priest)
        #print('b.shape: ', b.shape)
        biases += b / n
        #print('b: ', b)
    # return (weights, biases)
    # Now we have averaged weights and biases
    amodel = nn.Linear(2,1)
    amodel.load_state_dict(OrderedDict([
        ('weight', weights.get()),
        ('bias', biases.get())
    ]))
    return amodel 
    
#(weights, biases) = agregate_models([models['bob'], models['alice']])

amodel = agregate_models([models['bob'], models['alice']])

print('priest._objects', priest._objects)
# print('weights', weights.get())
# print('biases', biases.get())
print(amodel.state_dict())

    

priest._objects {62417679505: tensor([[1.9397, 1.0034]]), 85681731994: tensor([0.3291]), 51156512266: Parameter containing:
tensor([[1.9123, 0.9902]], requires_grad=True), 93104533621: Parameter containing:
tensor([0.5635], requires_grad=True), 8243136431: Parameter containing:
tensor([[1.9670, 1.0167]], requires_grad=True), 57408893389: Parameter containing:
tensor([0.0947], requires_grad=True)}
OrderedDict([('weight', tensor([[1.9397, 1.0034]])), ('bias', tensor([0.3291]))])


In [11]:
with th.no_grad():
    test_data = th.tensor([[1.,1],[0,1],[1,0],[0,0]])    
    test_target = get_synthetic_target(test_data)
    #test_data = test_data.send(bob)
    #test_target = test_target.send(bob)

    prediction = amodel.forward(test_data)#.get()    
    diff = prediction - test_target
    print('prediction: ', prediction)
    print('actual: ', test_target)
    print('test diff: ', diff)
    

prediction:  tensor([[3.2722],
        [1.3325],
        [2.2688],
        [0.3291]])
actual:  tensor([[3.],
        [1.],
        [2.],
        [0.]])
test diff:  tensor([[0.2722],
        [0.3325],
        [0.2688],
        [0.3291]])
